In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [ ]:
pwd

In [ ]:
import os    # 운영체제의 기능 사용
from selenium import webdriver    # 웹브라우저 조작, 자동화
from selenium.webdriver.firefox.options import Options    # 파이어폭스의 옵션사용?
from PIL import Image     # 이미지 처리 라이브러리 pillow(PIL)
from io import BytesIO    # 입출력,스트림 작업 / 바이트배열을 이진파일로 다룰 수 있게
from tqdm.notebook import tqdm    # 반복문 진행률 표시
import time    # 프로세스 정지(Delay)

In [ ]:
options = Options()        # 파이어폭스 기능 사용을 위한 options 인스턴스(객체) 생성
options.headless = True    # headless옵션 True 설정 : 브라우저를 보이지 않게

In [ ]:
driver = webdriver.Firefox()    # 오류발생시 다운받은 geckodriver path설정
driver.get("https://thebook.io/080228/application/")  # 실행할 url입력

In [ ]:
# 전체 페이지의 사이즈를 구하여 브라우저의 창 크기를 확대하고 스크린캡처를 합니다.
page_width = driver.execute_script('return document.body.parentNode.scrollWidth')
page_height = driver.execute_script('return document.body.parentNode.scrollHeight')

In [ ]:
print('page_width:', page_width); print('page_height:', page_height)

In [ ]:
driver.set_window_size(page_width, 3800)   # 브라우저 창 크기조절
png = driver.get_screenshot_as_png()       # 스크린샷(전체)을 찍고 png로 변환한 객체생성

In [ ]:
# 캡처할 영역 가져오기
element = driver.find_element_by_class_name('thebook-content.mdl-color--white.mdl-shadow--4dp.content.mdl-color-text--grey-800.mdl-cell.mdl-cell--12-col')

image_location = element.location   # 캡처할 영역의 x,y좌표 저장
image_size = element.size           # 캡처할 영역의 크기 저장

In [ ]:
print('image_location', image_location); print('image_size', image_size)

In [ ]:
im = Image.open(BytesIO(png))
im

In [ ]:
im = Image.open(BytesIO(png))    # 이전에 생성한 png객체를 byte형태로 오픈

# 좌표 계산
left = image_location['x']
top = image_location['y']
right = image_location['x'] + image_size['width']
bottom = image_location['y'] + image_size['height']

print(left, top, right, bottom, sep='\t')

In [ ]:
im = im.crop((left, top, right, bottom))
im.save('./image/Test.png')

In [ ]:
options = Options()
options.headless = True
driver = webdriver.Firefox()
driver.get("https://thebook.io/080228/application/")

num = 1    # 파일번호를 매기기 위한 변수
for i in tqdm(range(500)):    # 전체페이지수를 몰라 임의의 수 500설정
    # 전체 페이지의 사이즈를 구하여 브라우저의 창 크기를 확대하고 스크린캡처를 합니다.
#   page_width = driver.execute_script('return document.body.parentNode.scrollWidth')
#   page_height = driver.execute_script('return document.body.parentNode.scrollHeight')
    driver.set_window_size(1200, 3800)
    png = driver.get_screenshot_as_png()

    # 특정 element의 위치와 크기를 구합니다.
    element = driver.find_element_by_class_name('thebook-content.mdl-color--white.mdl-shadow--4dp.content.mdl-color-text--grey-800.mdl-cell.mdl-cell--12-col')
    image_location = element.location
    image_size = element.size

    # 이미지를 element의 위치에 맞춰서 crop 하고 저장합니다.
    im = Image.open(BytesIO(png))
    left = image_location['x']
    top = image_location['y']
    right = image_location['x'] + image_size['width']
    if image_size['height'] == 3800:
        bottom = 1200
    else:
        bottom = image_location['y'] + image_size['height']
    im = im.crop((left, top, right, bottom))
    im.save('./image/Modu_Deep_{}.png'.format(num))

    try:
        driver.find_element_by_id('next-page').click()
        time.sleep(0.5)
        num += 1    # 파일 번호 추가
            
    except:    # 광고페이지로 넘어갔을 시
        os.remove('./image/Modu_Deep_{}.png'.format(num))  # 저장한이미지(광고) 삭제
        driver.back()    # 이전페이지로
        driver.find_element_by_id('next-page').click()     # 다음페이지 버튼 클릭
        time.sleep(0.5)